# Randomized Benchmarking

Here we will show how to calculate the average Clifford gate fidelity using randomized benchmarking.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import qiskit
import qiskit.quantum_info as qqi
import scipy.optimize
from qbraid import QbraidProvider

from equal1_demo import qbraid_helper

We will use the qiskit Quantum information library to build up random Clifford benchmarking circuits.
In random Clifford Benchmarking we uniformly at random sample a sequence of Clifford gates $(C_1, C_2, C_3, \ldots, C_d)$
The idea is to apply this sequence of Clifford gates followed by an additional gate $C_{d+1}$ which inverts the action of the previous gates $I = C_{d+1} . \Pi_{i}^{d} C_i$ . 
If the initial state was $\ket{0}$ then the final state in a noiseless system should be $P(\ket{0}) = 1.0$. 
Instead as the sequences of gates get longer, $d$, the effect of noise causes a decay of the value of $P(\ket{0})$, the rate of this decay characterizes the average error per clifford gate. 

* https://arxiv.org/abs/1009.3639
* https://arxiv.org/abs/1109.6887

Each Clifford gate can be implemented by a subset of Clifford gates, Qiskit uses $X$, $Z$, $H$, $S$, $S^\dagger$.


In [ ]:
a_random_clifford = qqi.random_clifford(1, seed=39)
random_clifford_gates = a_random_clifford.to_circuit()
random_clifford_gates.draw("mpl")

We will now define a function that will build a benchmarking circuit for a given qubit and depth.

In [ ]:
circuit_meta[1]

In [ ]:
circuits[1].draw("mpl")

Now we want to run the circuit on the device:

In [ ]:
provider = QbraidProvider()
device = "equal1_simulator"
noise_model = "bell1-6"
device = provider.get_device(device)
device.profile.model_dump()


The Equal1 compiler will take a benchmarking circuit such as this, however, it tries to simplify the circuit. 
The benchmarking sequence has the property that $I = C_{d+1} . \Pi_{i}^{d} C_i$  and so the compiler will simplify it to a circuit with no gates! 

We can see this by setting `optimization_level=1`. 

In [ ]:
result, compiled = qbraid_helper.run_circuit(
    circuits[1],
    device,
    shots=shots,
    noise_model=noise_model,
    optimization_level=1,
    simulation_platform="CPU",
    get_probabilities=True,
    return_transpiled_circuits=True
)


In [ ]:
compiled[0].draw("mpl", scale=0.7, fold=100)


We will need to set the compiler optimization level to be 0, this will convert the gates to our native gate set ${R_x(\frac{\pi}{2}), R_z(\theta), CZ}$, but will not attempt to simplify the circuit. 

In [ ]:
print(f"Sending {len(circuits)} circuits to the device...")
results = qbraid_helper.run_circuit(
    circuits,
    device,
    shots=shots,
    noise_model=noise_model,
    optimization_level=0, # prevent compiler from simplifying the circuit to nothing
    simulation_platform="CPU",
    get_probabilities=True,
)


In [ ]:
results[:5]

In [ ]:
n_reps = len({m["repetition"] for m in circuit_meta})
n_depths = len({m["depth"] for m in circuit_meta})
probabilities = np.zeros((n_reps, n_depths), dtype=float)

for result, meta in zip(results, circuit_meta):
    probabilities[meta["repetition"], meta["depth_index"]] = result.get("0", 0)


In [ ]:
mean_probability = probabilities.mean(axis=0)

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.plot(depths, probabilities.T, marker="o", linestyle="none")
ax.plot(depths, mean_probability)
ax.set_xlabel("Circuit Depth")

In [ ]:
def decay_curve(m, A, alpha, B=0):
    return A * (alpha**m) + B


In [ ]:
fit = scipy.optimize.curve_fit(
    decay_curve,
    depths,
    mean_probability,
    p0=[0.99, 0.1, 0.5],
)[0][1]

print(f"Average Clifford gate fidelity on qubit {qubit_to_benchmark} = {fit}")
